# Foodies Table Exploration

### Utils

---

(Todo: move to python file)

In [18]:
import pandas as pd
def read_csv(table_name):
    return pd.read_csv("data/foodies_table_%s.csv"%table_name)

## Ranking Ideas for section

1. Section with most food. (Hypothesis: If there are many food in that section that there should be high demand in that type of section, thus leads to more user clicking)
2. Section with restaurant opening a lot. (Hypothesis: If those restaurant open hours is higher, then it shows higher demand for the customer. Less opening hour shows a more niche type of restaurant. (Which would only please few people)

## Ranking strategy

1. Put (the one we believe are) the best restaurants at the top.
2. Make the top as diverse as possible. (Exploration)
    1. If we have japanese food in the first few ones that we might opt to select another type of restaurant.

## Ranking Ideas for each restaurant

---

Content Based (based on content of each restaurant) No user data require just res data
1. Shop payment type (ยิ่งเยอะยิ่งดี)
2. Location ติด bts mrt ยิ่งเยอะยิ่งดี
3. Shop_price. Let's say ยิ่งถูกยิ่งดีไปก่อน ยังไม่คำนึงภถึงคุณภาพ. (จริงๆควรเป็น quality/price)

Content Based (with user data)
1. Review (count, taste score, value_score, service_score, appearance score, overall_score)
2. Location of restaurant match with user

Collaborative Filtering (rank based on other user)
1. Clustered Food then match user to those cluster.
2. Also have to find way to identify user food preference. 
    1. Member_like might help
    2. Member province (if data is available)

## Feature that would be good to have

1. Member address (especially lat lng) We could compare that with restaurant data.
2. Member_like, follow
3. Reviews. (I'm assuming we don't have those yet. Would be good to have although I think we need lot of it to be valuable.)

## Future Feature Ideas

1. Image quality score: รูปภาพไหนที่สวยจะมีคะแนนสวยเยอะ เอามาช่วย ranking
    - Where would it come from? There are two ways I can think of.
        1. Get feedback data from user. Which picture have been cilcked the most should somehow be consider as nice picture.
        2. Tag the quality score by ourselves in some images then use those as a training set.
2. Social media crawler.
    I saw a field that link to social media. We could make a crawler to download data from those pages and analyze quality score of restaurants.

## Going through features we could use

In [41]:
# Exploring potential table that we can use for ranking based on content data.
shop = read_csv("shop")
review = read_csv("review")
food_nationality = read_csv("food_nationality")

In [103]:
print(shop.shape)
shop.columns
# Interesting features includes (I'm imagining all the data are real)
features = [
    'description', # Maybe longer -> better?? Or descrption with some certain keywords has better score. (NLP)
    'tel', 'website', 'line', 'facebook', 'instagram', # have vs doesn't have
    'no_room', # more = better?
    'no_floor', # higher = better? (will have low weight)
    'lat', 'lng', # For location score
    'shop_price_from', 'shop_price_to', # Price features
    'has_carpark', # has = better
    'worktime', # more opening hour ~~ better
    'last_order', # more late ~~ better
]

(65040, 34)


In [107]:
# Let's see how many of them looks ready.
shop[features].notnull().sum()

description           40
tel                40198
website             6786
line                4876
facebook           16679
instagram           4464
no_room               40
no_floor              40
lat                57358
lng                57358
shop_price_from    65040
shop_price_to      65040
has_carpark        65033
worktime           44116
last_order         65040
dtype: int64

In [111]:
# Look's reasonable. I think we could start with this set of data on this table only.

## Check from other table

---

These are all potentially good features for ranking

From the exporation, we found out that there aren't so many shop_id which has these features in the table yet.

In [96]:
def analyze_table(table_name):
    df = read_csv(table_name)
    print(df.shape)
    # Check how many table has this feature
    df2 = read_csv("shop_"+table_name)
    print("Unique Shop_id with this featuers: %d"%df2.shop_id.unique().shape[0])
    return df.head()

In [97]:
# Near BTS/MRT = Better
analyze_table("shop_location")

(68, 10)
Unique Shop_id with this featuers: 33


,id,parent_id,name_th,name_en,order_on,status,created_on,created_by,updated_on,updated_by
0,1,0,ใกล้บีทีเอส,Near BTS,1,Y,2019-02-11 22:13:56,-,0000-00-00 00:00:00,-
1,2,0,ใกล้เอ็มอาร์ที,Near MRT,2,Y,2019-02-11 22:13:56,-,0000-00-00 00:00:00,-
2,3,0,เขาใหญ่,Khao Yai,3,Y,2019-02-11 22:13:56,-,0000-00-00 00:00:00,-
3,4,0,บึงกาฬ,Bueng Kan,4,Y,2019-02-11 22:13:56,-,0000-00-00 00:00:00,-
4,5,0,ปริมณฑล,Perimeter,5,Y,2019-02-11 22:13:56,-,0000-00-00 00:00:00,-


In [98]:
# More variety of payment option = better
analyze_table("shop_pay")

(3, 9)
Unique Shop_id with this featuers: 44


,id,name_th,name_en,order_on,status,created_on,created_by,updated_on,updated_by
0,1,เงินสด,Cash,1,Y,2019-02-12 00:34:57,-,0000-00-00 00:00:00,-
1,2,บัตรเครดิต,Credit Card,2,Y,2019-02-12 00:34:58,-,0000-00-00 00:00:00,-
2,3,พร้อมเพย์,Prompt Pay,3,Y,2019-02-12 00:34:58,-,0000-00-00 00:00:00,-


In [99]:
# More service -> better (?)
analyze_table("shop_service")

(14, 9)
Unique Shop_id with this featuers: 33


,id,name_th,name_en,order_on,status,created_on,created_by,updated_on,updated_by
0,1,จำหน่ายไวน์,Sell wine,1,Y,2019-02-11 21:46:28,-,0000-00-00 00:00:00,-
1,2,ซื้อกลับบ้าน,Take home,2,Y,2019-02-11 21:46:28,-,0000-00-00 00:00:00,-
2,3,ดนตรีสด,Live music,3,Y,2019-02-11 21:46:28,-,0000-00-00 00:00:00,-
3,4,ถ่ายทอดสดกีฬา,Live sports,4,Y,2019-02-11 21:46:28,-,0000-00-00 00:00:00,-
4,5,บริการจัดส่ง,Delivery service,5,Y,2019-02-11 21:46:28,-,0000-00-00 00:00:00,-


In [100]:
# Price features
read_csv("shop_price")

,id,name_th,name_en,order_on,status,created_on,created_by,updated_on,updated_by
0,1,200 - 500,200 - 500,1,Y,2019-02-12 00:37:41,-,0000-00-00 00:00:00,-
1,2,"501 - 1,000","501 - 1,000",2,Y,2019-02-12 00:37:41,-,0000-00-00 00:00:00,-
2,3,"1,001 - 2,500","1,001 - 2,500",3,Y,2019-02-12 00:37:41,-,0000-00-00 00:00:00,-
3,4,"2,501 ขึ้นไป","More Than 2,501",4,Y,2019-02-12 00:37:41,-,0000-00-00 00:00:00,-


In [101]:
# Can put score base on each type of food (based on user preference).
analyze_table("food_nationality")

(49, 10)
Unique Shop_id with this featuers: 36


,id,parent_id,name_th,name_en,order_on,status,created_on,created_by,updated_on,updated_by
0,1,0,นานาชาติ,International,1,Y,2019-02-11 21:52:05,-,0000-00-00 00:00:00,-
1,2,0,เอเชีย,Asia,2,Y,2019-02-11 21:52:05,-,0000-00-00 00:00:00,-
2,3,0,ออสเตรเลีย,Australia,3,Y,2019-02-11 21:52:05,-,0000-00-00 00:00:00,-
3,4,0,ยุโรป,Europe,4,Y,2019-02-11 21:52:06,-,0000-00-00 00:00:00,-
4,5,0,อเมริกา,America,5,Y,2019-02-11 21:52:06,-,0000-00-00 00:00:00,-


## Let's not use these for now

From the look of these data, I think we still need more for it to be usable. 

I would expect at least half of the shop to have the feature in table to be able to use that table.